dataset : Multimodal Physiological Monitoring During Virtual Reality Piloting Tasks v1.0.0 (physionet.org)
download in dataDirDev

In [ ]:
import os
import random
import load
import DownSampling
dataDirDev = "." + os.sep + "dataPackage"
featDirDevs = "." + os.sep + "data_feats" + os.sep + "combined"
featDirDev = "." + os.sep + "data_feats"
modelDirDev = "." + os.sep + "model_eval"

expType = "task-ils"
dataDir = dataDirDev + os.path.sep + expType
dirs: list[list[str]] = []

subjDirs = [f.path for f in os.scandir(dataDir) if f.is_dir()]
for subjd in subjDirs:
    if 'sub-cp003' in subjd:
        continue
    sessDirs = [f.path for f in os.scandir(subjd) if f.is_dir()]
    for sessd in sessDirs:
        runDirs = [f.path for f in os.scandir(sessd) if f.is_dir()]
        for rund in runDirs:
            dataFiles = [f.path for f in os.scandir(rund) if (f.is_file() and not f.name.startswith('.'))]
            tokens = rund.split(os.path.sep)
            subj = tokens[-3]
            sess = tokens[-2]
            level = tokens[-1].split('_')[0]
            run = tokens[-1].split('_')[1]
            dirs.append([subj, sess, level, run])

pctHoldout = 25
numDirs = len(dirs)
numVal = int(numDirs * float(pctHoldout) / 100)
numTrain = numDirs - numVal
random.shuffle(dirs)
trainSet = dirs[:numTrain]
valSet = dirs[numTrain:]

aggFeatFilePath = featDirDev + os.path.sep + 'devSubjsFeatMat_vae.csv'
featMatDF = afm.getFeatureMatrixSingleFile(dataDirDev, expType, dirs, aggFeatFilePath)

featFiles = [f.path for f in os.scandir(featDirDev) if (f.is_file() and not f.name.startswith('.'))]
for DIR in dirs:
    subj = DIR[0]
    sess = DIR[1]
    level = DIR[2]
    run = DIR[3]
    featFile = featDirDev + os.path.sep + subj + "_" + sess + "_" + level + "-" + run + "_" + "combined_data.csv"
    if featFile in featFiles:
        continue
    eye_data, eda_data, emg_data, resp_data, acc_data, ecg_data = None, None, None, None, None, None
    try:
        eye_data = load.load_data(dataDir, subj, sess, expType, 'lslhtcviveeye', level, run)
        eda_data = load.load_data(dataDir, subj, sess, expType, 'lslshimmereda', level, run)
        emg_data = load.load_data(dataDir, subj, sess, expType, 'lslshimmeremg', level, run)
        resp_data = load.load_data(dataDir, subj, sess, expType, 'lslshimmerresp', level, run)
        ecg_data = load.load_data(dataDir, subj, sess, expType, 'lslshimmerecg', level, run)
        # ace_data = load.load_data(dataDir, subj, sess, expType, 'lslrespitrace', level, run)
    except Exception:
        continue
    combined_data = DownSampling.down_sampling_v2(eye_data, eda_data, emg_data, resp_data, ecg_data)
    combined_data.to_csv(
        featDirDev + os.path.sep + subj + "_" + sess + "_" + level + "-" + run + "_" + "combined_data.csv",
        index=False)
